# imgtools.mstools

<a id='notes'></a>

## Notes

The imgtools.mstools package contains tasks for working with STIS, NICMOS, ACS, and WFC3 data. Some tasks are 'extensions" of existing tasks in the STSDAS system, and support other instruments/file formats as well.


Contents:

* [mscombine](#mscombine)

<a id='mscombine'></a>

## mscombine

**Please review the [Notes](#notes) section above before running any examples in this notebook**

The original `mscombine` IRAF task performed image combination of several `SCI` exetensions of HST data while allowing the user to reject specified `DQ` bits. Additionally, the user could choose to combine the stack using the average or the median.  

This `mscombine` alternative uses `numpy` masked arrays to avoid using flagged pixels in the `DQ` array.  In this simple example, we average-combine several full-frame WFC3/UVIS images.

In [1]:
# Standard Imports
import glob
import numpy as np

# Astronomy Specific Imports
from astropy.io import fits

In [2]:
# Get the data
test_data = glob.glob('/eng/ssb/iraf_transition/test_data/mscombine/*_blv_tmp.fits')

In [3]:
# Create masked arrays
masked_arrays_ext1, masked_arrays_ext2, masked_arrays_ext4, masked_arrays_ext5 = [], [], [], []
for filename in test_data:
    with fits.open(filename) as hdulist:
        
            # For UVIS chip 2
            mask_ext3 = np.zeros(hdulist[3].data.shape)
            mask_ext3[np.where(hdulist[3].data != 0)] = 1
            masked_arrays_ext1.append(np.ma.masked_array(hdulist[1].data, mask=mask_ext3))
            masked_arrays_ext2.append(np.ma.masked_array(hdulist[2].data, mask=mask_ext3))

            # For UVIS chip 1
            mask_ext6 = np.zeros(hdulist[6].data.shape)
            mask_ext6[np.where(hdulist[6].data != 0)] = 1
            masked_arrays_ext4.append(np.ma.masked_array(hdulist[4].data, mask=mask_ext6))
            masked_arrays_ext5.append(np.ma.masked_array(hdulist[5].data, mask=mask_ext6))

In [4]:
# Average-combine SCI arrays
comb_ext1 = np.ma.mean(masked_arrays_ext1, axis=0).data
comb_ext4 = np.ma.mean(masked_arrays_ext4, axis=0).data

In [5]:
# Propoagate uncertainties for ERR arrays
weight_image_ext1 = np.zeros((2051, 4096))
weight_image_ext4 = np.zeros((2051, 4096))
for array in masked_arrays_ext1:
    mask = array.mask
    weight_image_ext1[np.where(mask == False)] += 1.0
for array in masked_arrays_ext4:
    mask = array.mask
    weight_image_ext4[np.where(mask == False)] += 1.0
masked_arrays_ext2_squared = [(item * (1/weight_image_ext1))**2 for item in masked_arrays_ext2]
masked_arrays_ext5_squared = [(item * (1/weight_image_ext4))**2 for item in masked_arrays_ext5]
comb_ext2 = np.sqrt(np.ma.sum(masked_arrays_ext2_squared, axis=0)).data
comb_ext5 = np.sqrt(np.ma.sum(masked_arrays_ext5_squared, axis=0)).data

/Users/bourque/anaconda2/envs/astroconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: RuntimeWarning: divide by zero encountered in true_divide
/Users/bourque/anaconda2/envs/astroconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: RuntimeWarning: divide by zero encountered in true_divide


In [6]:
# Create empty DQ arrays
comb_ext3 = np.zeros((2051, 4096))
comb_ext6 = np.zeros((2051, 4096))

In [7]:
# Build and save the combined file, using the first final for the header
hdu0 = fits.PrimaryHDU(header=fits.getheader(test_data[0], 0))
hdu1 = fits.ImageHDU(comb_ext1, header=fits.getheader(test_data[0], 0))
hdu2 = fits.ImageHDU(comb_ext2, header=fits.getheader(test_data[0], 1))
hdu3 = fits.ImageHDU(comb_ext3, header=fits.getheader(test_data[0], 2))
hdu4 = fits.ImageHDU(comb_ext4, header=fits.getheader(test_data[0], 3))
hdu5 = fits.ImageHDU(comb_ext5, header=fits.getheader(test_data[0], 4))
hdu6 = fits.ImageHDU(comb_ext6, header=fits.getheader(test_data[0], 5))
hdulist = fits.HDUList([hdu0, hdu1, hdu2, hdu3, hdu4, hdu5, hdu6])
hdulist.writeto('test.fits')

For questions or comments please see [our github page](https://github.com/spacetelescope/stak).  We encourage and appreciate user feedback.